In [ ]:
!pip install transformers
!pip install datasets
# !pip install -q transformers accelerate
# !pip install sentence_transformers
# !pip install faiss-cpu
!pip install cleantext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset
from sklearn.preprocessing import LabelEncoder
import cleantext as c
import nltk
import numpy as np
from nltk.corpus import stopwords
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# Download the stopwords once if not already downloaded
# nltk.download('stopwords')
# stop_words = set(stopwords.words('english'))

In [ ]:
# data = load_dataset("lhoestq/demo1")

In [ ]:
dataset = load_dataset("csv", data_files="/content/drive/MyDrive/data.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Sentence', 'Sentiment'],
        num_rows: 5842
    })
})

In [ ]:
dataset['train'][0]

{'Sentence': "The GeoSolutions technology will leverage Benefon 's GPS solutions by providing Location Based Search Technology , a Communities Platform , location relevant multimedia content and a new and powerful commercial model .",
 'Sentiment': 'positive'}

In [ ]:
df = dataset['train']

In [ ]:
df = df.to_pandas()

In [ ]:
df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5842 entries, 0 to 5841
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentence   5842 non-null   object
 1   Sentiment  5842 non-null   object
dtypes: object(2)
memory usage: 91.4+ KB


In [ ]:
def label_sentiment(row):
    if row == "positive":
        return 2
    elif row == "negative":
        return 0
    elif row == "neutral":
        return 1
    else:
        raise ValueError("Unexpected sentiment value: {}".format(row))

# Example usage with a DataFrame column
df['Sentiment'] = df['Sentiment'].apply(label_sentiment)


In [ ]:
df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,2
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",0
2,"For the last quarter of 2010 , Componenta 's n...",2
3,According to the Finnish-Russian Chamber of Co...,1
4,The Swedish buyout firm has sold its remaining...,1


In [ ]:
df['Sentiment'].value_counts(normalize=True)*100

,proportion
Sentiment,
1,53.577542
2,31.701472
0,14.720986


In [ ]:
# Clean, remove stopwords, and lowercase text
cleaned_data = [
c.clean(sentence, stopwords=True,stemming=True).lower() for sentence in df['Sentence']
]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Remove special characters using regex
clean_sentence = [(re.sub(r'[^a-zA-Z\s]', '', data)) for data in cleaned_data]

In [ ]:
clean_sentence

['the geosolut technolog leverag benefon s gp solut provid locat base search technolog  commun platform  locat relev multimedia content new power commerci model ',
 'esi lows   bk real possibl',
 'for last quarter   componenta s net sale doubl eurm eurm period year earlier  move zero pretax profit pretax loss eurm ',
 'accord finnishrussian chamber commerc  major construct compani finland oper russia ',
 'the swedish buyout firm sold remain  percent stake  almost eighteen month take compani public finland ',
 'spi surpris see green close',
 'shell  billion bg deal meet sharehold skeptic',
 'ssh commun secur corp stock exchang releas octob    at  pm the compani updat full year outlook estim result remain loss full year ',
 'kone s net sale rose   yearonyear first nine month  ',
 'the stockmann depart store total floor space  squar metr stockmann s invest project price tag eur  million ',
 'circul revenu increas   finland   sweden  ',
 'sap q disappoint softwar licens down real problem c

In [ ]:
df['Sentence'] = clean_sentence

In [ ]:
df

,Sentence,Sentiment
0,the geosolut technolog leverag benefon s gp so...,2
1,esi lows bk real possibl,0
2,for last quarter componenta s net sale doubl...,2
3,accord finnishrussian chamber commerc major c...,1
4,the swedish buyout firm sold remain percent s...,1
...,...,...
5837,rise cost forc packag produc huhtamaki axe jo...,0
5838,nordic walk first use summer train method cros...,1
5839,accord ship compani vike line eu decis signif...,1
5840,in build home improv trade sale decreas eur...,1


In [ ]:
dataset = Dataset.from_pandas(df)

In [ ]:
dataset

Dataset({
    features: ['Sentence', 'Sentiment'],
    num_rows: 5842
})

In [ ]:
# Split into 80% train, 20% test
split_dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['Sentence', 'Sentiment'],
        num_rows: 4673
    })
    test: Dataset({
        features: ['Sentence', 'Sentiment'],
        num_rows: 1169
    })
})

In [ ]:
# Further split for validation
train_valid_split = split_dataset['train'].train_test_split(test_size=0.1)
train_valid_split.set_format('torch')
final_dataset = {
    'train': train_valid_split['train'],
    'validation': train_valid_split['test'],
    'test': split_dataset['test']
}

# Check the final splits
print(final_dataset)

{'train': Dataset({
    features: ['Sentence', 'Sentiment'],
    num_rows: 4205
}), 'validation': Dataset({
    features: ['Sentence', 'Sentiment'],
    num_rows: 468
}), 'test': Dataset({
    features: ['Sentence', 'Sentiment'],
    num_rows: 1169
})}


In [ ]:
final_dataset['test']

Dataset({
    features: ['Sentence', 'Sentiment'],
    num_rows: 1169
})

In [ ]:
# Load a tokenizer and model
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def tokenize(batch):
    return tokenizer(batch['Sentence'], padding="max_length", truncation=True, max_length=128)

# Tokenize each split separately
tokenized_dataset = {
    split: data.map(tokenize, batched=True)
    for split, data in final_dataset.items()
}

# Rename the label column in each split
tokenized_dataset = {
    split: data.rename_column("Sentiment", "labels")
    for split, data in tokenized_dataset.items()
}

# Remove the "Sentence" column in each split
tokenized_dataset = {
    split: data.remove_columns(["Sentence"])
    for split, data in tokenized_dataset.items()
}


Map:   0%|          | 0/4205 [00:00<?, ? examples/s]

Map:   0%|          | 0/468 [00:00<?, ? examples/s]

Map:   0%|          | 0/1169 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

{'train': Dataset({
     features: ['labels', 'input_ids', 'attention_mask'],
     num_rows: 4205
 }),
 'validation': Dataset({
     features: ['labels', 'input_ids', 'attention_mask'],
     num_rows: 468
 }),
 'test': Dataset({
     features: ['labels', 'input_ids', 'attention_mask'],
     num_rows: 1169
 })}

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation']
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,0.644447
2,0.609700,0.561680
3,0.609700,0.570397
4,0.383500,0.807957
5,0.383500,0.810605
6,0.260200,0.843147
7,0.260200,0.983252
8,0.222400,1.115744
9,0.222400,1.079740
10,0.190800,1.317873


Epoch,Training Loss,Validation Loss
1,No log,0.644447
2,0.609700,0.561680
3,0.609700,0.570397
4,0.383500,0.807957
5,0.383500,0.810605
6,0.260200,0.843147
7,0.260200,0.983252
8,0.222400,1.115744
9,0.222400,1.079740
10,0.190800,1.317873


TrainOutput(global_step=5260, training_loss=0.23173877404216578, metrics={'train_runtime': 2159.7748, 'train_samples_per_second': 38.939, 'train_steps_per_second': 2.435, 'total_flos': 5531959607731200.0, 'train_loss': 0.23173877404216578, 'epoch': 20.0})

In [ ]:
# Evaluate on the validation set
validation_results = trainer.evaluate(tokenized_dataset['validation'])
print("Validation Results:", validation_results)

# Evaluate on the test set
test_results = trainer.evaluate(tokenized_dataset['test'])
print("Test Results:", test_results)

Validation Results: {'eval_loss': 1.8372166156768799, 'eval_runtime': 3.1348, 'eval_samples_per_second': 149.292, 'eval_steps_per_second': 9.57, 'epoch': 20.0}
Test Results: {'eval_loss': 1.7390782833099365, 'eval_runtime': 8.3172, 'eval_samples_per_second': 140.553, 'eval_steps_per_second': 8.897, 'epoch': 20.0}


In [ ]:
results = trainer.predict(tokenized_dataset['test'])

In [ ]:
results

PredictionOutput(predictions=array([[ 2.9641376, -5.2447324,  1.8123956],
       [-3.827163 , -2.8635247,  7.1633787],
       [-2.286205 ,  6.572859 , -3.7933757],
       ...,
       [-2.3371263,  6.760572 , -3.9716582],
       [-1.9861785,  6.7152953, -4.290306 ],
       [ 5.2599173, -2.5714195, -3.025655 ]], dtype=float32), label_ids=array([1, 2, 1, ..., 1, 1, 0]), metrics={'test_loss': 1.7390782833099365, 'test_runtime': 8.4774, 'test_samples_per_second': 137.895, 'test_steps_per_second': 8.729})

In [ ]:
# Extract logits
logits = results.predictions

# Convert logits to predicted class labels
predicted_classes = np.argmax(logits, axis=1)

# Print or analyze the predicted class labels
print(predicted_classes)

[0 2 1 ... 1 1 0]


In [ ]:
predicted_classes

array([0, 2, 1, ..., 1, 1, 0])

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# True labels (from the test set)
true_labels = results.label_ids

# Predicted class labels
predicted_classes = np.argmax(results.predictions, axis=1)

# Compute accuracy
accuracy = accuracy_score(true_labels, predicted_classes)

# Compute precision, recall, and F1-score for each class
precision, recall, f1, support = precision_recall_fscore_support(true_labels, predicted_classes, average=None)

# Display results for each class
print(f"Overall Accuracy: {accuracy:.4f}")
for i, (p, r, f, s) in enumerate(zip(precision, recall, f1, support)):
    print(f"Class {i} - Precision: {p:.4f}, Recall: {r:.4f}, F1 Score: {f:.4f}, Support: {s}")


Overall Accuracy: 0.7331
Class 0 - Precision: 0.4202, Recall: 0.4463, F1 Score: 0.4329, Support: 177
Class 1 - Precision: 0.7738, Recall: 0.7873, F1 Score: 0.7805, Support: 630
Class 2 - Precision: 0.8294, Recall: 0.7790, F1 Score: 0.8034, Support: 362
